In [1]:
import glob
import pandas as pd
import sqlite3
from pymongo import MongoClient
from sodapy import Socrata
import requests
from math import ceil
import glob

In [2]:
#connecting to the sqlite database
conn = sqlite3.connect('health_air.db')
#connecting to the MongoDB Database
mongo_client = MongoClient()
#creating the MongoDB
mydb = mongo_client['health_air_mongo']

In [3]:
#function to add a table to the database
def add_table_to_db(table_name, df, connection_db):
    cur = conn.cursor()
    df.to_sql(name=table_name, if_exists='replace', con=conn)
    return(cur.execute(f"""SELECT * FROM {table_name};""").fetchall())

In [4]:
def add_collection_to_mongo (collection_name, df, mongo_db_conn = mydb):
    mongo_db_conn[collection_name].insert_many(df.to_dict('records'))
    return print(f'{collection_name} stored in the MongoDB database named {mydb}')

# ETL of [Asthma data](https://chronicdata.cdc.gov/Chronic-Disease-Indicators/U-S-Chronic-Disease-Indicators-Asthma/us8e-ubyj)

In [5]:
# Instruction from the CDC website to extract data
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("chronicdata.cdc.gov", None)

# Return as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get_all("us8e-ubyj")

# Convert to pandas DataFrame
asthma_df = pd.DataFrame.from_records(results)

In [6]:
asthma_df

,yearstart,yearend,locationabbr,locationdesc,datasource,topic,question,datavaluetype,datavalue,datavaluealt,...,topicid,questionid,datavaluetypeid,stratificationcategoryid1,stratificationid1,datavalueunit,lowconfidencelimit,highconfidencelimit,datavaluefootnotesymbol,datavaluefootnote
0,2012,2012,AL,Alabama,NVSS,Asthma,Asthma mortality rate,Number,37,37,...,AST,AST4_1,NMBR,GENDER,GENF,NaN,NaN,NaN,NaN,NaN
1,2014,2014,AL,Alabama,NVSS,Asthma,Asthma mortality rate,Number,22,22,...,AST,AST4_1,NMBR,GENDER,GENM,NaN,NaN,NaN,NaN,NaN
2,2011,2011,AZ,Arizona,NVSS,Asthma,Asthma mortality rate,Number,29,29,...,AST,AST4_1,NMBR,GENDER,GENM,NaN,NaN,NaN,NaN,NaN
3,2015,2015,CT,Connecticut,NVSS,Asthma,Asthma mortality rate,Number,34,34,...,AST,AST4_1,NMBR,GENDER,GENF,NaN,NaN,NaN,NaN,NaN
4,2011,2011,FL,Florida,NVSS,Asthma,Asthma mortality rate,Number,54,54,...,AST,AST4_1,NMBR,GENDER,GENM,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64714,2011,2011,WI,Wisconsin,BRFSS,Asthma,Pneumococcal vaccination among noninstitutiona...,Age-adjusted Prevalence,NaN,NaN,...,AST,AST6_2,AGEADJPREV,GENDER,GENM,%,NaN,NaN,****,Sample size of denominator and/or age group fo...
64715,2016,2016,TX,Texas,BRFSS,Asthma,Pneumococcal vaccination among noninstitutiona...,Age-adjusted Prevalence,NaN,NaN,...,AST,AST6_2,AGEADJPREV,RACE,MRC,%,NaN,NaN,****,Sample size of denominator and/or age group fo...
64716,2019,2019,WI,Wisconsin,BRFSS,Asthma,Pneumococcal vaccination among noninstitutiona...,Age-adjusted Prevalence,NaN,NaN,...,AST,AST6_2,AGEADJPREV,RACE,OTH,%,NaN,NaN,****,Sample size of denominator and/or age group fo...
64717,2014,2014,PR,Puerto Rico,BRFSS,Asthma,Pneumococcal vaccination among noninstitutiona...,Crude Prevalence,NaN,NaN,...,AST,AST6_2,CRDPREV,GENDER,GENM,%,NaN,NaN,****,Sample size of denominator and/or age group fo...


In [7]:
#printing the names of the dataframe columns
asthma_df.columns

Index(['yearstart', 'yearend', 'locationabbr', 'locationdesc', 'datasource',
       'topic', 'question', 'datavaluetype', 'datavalue', 'datavaluealt',
       'stratificationcategory1', 'stratification1', 'locationid', 'topicid',
       'questionid', 'datavaluetypeid', 'stratificationcategoryid1',
       'stratificationid1', 'datavalueunit', 'lowconfidencelimit',
       'highconfidencelimit', 'datavaluefootnotesymbol', 'datavaluefootnote'],
      dtype='object')

In [8]:
# printing the question the data are answering
asthma_df['question'].unique()

array(['Asthma mortality rate',
       'Emergency department visit rate for asthma',
       'Hospitalizations for asthma',
       'Current asthma prevalence among adults aged >= 18 years',
       'Asthma prevalence among women aged 18-44 years',
       'Influenza vaccination among noninstitutionalized adults aged 18-64 years with asthma',
       'Influenza vaccination among noninstitutionalized adults aged >= 65 years with asthma',
       'Pneumococcal vaccination among noninstitutionalized adults aged 18-64 years with asthma',
       'Pneumococcal vaccination among noninstitutionalized adults aged >= 65 years with asthma'],
      dtype=object)

In [9]:
#list containing the questions of interest for our project
questions_OI = ['Current asthma prevalence among adults aged >= 18 years']


In [10]:
#retrieving the unique ID per question so that we address eventual spelling mistakes
questionids_list=[]
for question in questions_OI:
    questionids_list += list(asthma_df[asthma_df['question']==question]['questionid'].unique())
print(questionids_list)

['AST1_1']


In [11]:
# filtering the dataset for the questions of interest
filtered_asthma_df = asthma_df[asthma_df['questionid'].isin(questionids_list)]
filtered_asthma_df

,yearstart,yearend,locationabbr,locationdesc,datasource,topic,question,datavaluetype,datavalue,datavaluealt,...,topicid,questionid,datavaluetypeid,stratificationcategoryid1,stratificationid1,datavalueunit,lowconfidencelimit,highconfidencelimit,datavaluefootnotesymbol,datavaluefootnote
19,2017,2017,IA,Iowa,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,Crude Prevalence,12,12,...,AST,AST1_1,CRDPREV,GENDER,GENF,%,10.7,13.3,NaN,NaN
20,2016,2016,MO,Missouri,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,Crude Prevalence,7,7,...,AST,AST1_1,CRDPREV,GENDER,GENM,%,5.8,8.5,NaN,NaN
22648,2016,2016,AZ,Arizona,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,Crude Prevalence,6.7,6.7,...,AST,AST1_1,CRDPREV,RACE,HIS,%,5,9,NaN,NaN
22649,2019,2019,AL,Alabama,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,Crude Prevalence,9.5,9.5,...,AST,AST1_1,CRDPREV,OVERALL,OVR,%,8.6,10.5,NaN,NaN
22650,2012,2012,CO,Colorado,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,Crude Prevalence,7.1,7.1,...,AST,AST1_1,CRDPREV,GENDER,GENM,%,6.2,8.1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37543,2016,2016,NM,New Mexico,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,Crude Prevalence,11.4,11.4,...,AST,AST1_1,CRDPREV,RACE,WHT,%,9.9,13.2,NaN,NaN
37544,2011,2011,RI,Rhode Island,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,NaN,NaN,...,AST,AST1_1,AGEADJPREV,RACE,MRC,%,NaN,NaN,****,Sample size of denominator and/or age group fo...
37547,2019,2019,ID,Idaho,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,7.9,7.9,...,AST,AST1_1,AGEADJPREV,GENDER,GENM,%,6.4,9.7,NaN,NaN
37550,2014,2014,MO,Missouri,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,Crude Prevalence,9.7,9.7,...,AST,AST1_1,CRDPREV,OVERALL,OVR,%,8.7,10.8,NaN,NaN


In [12]:
#checking if column 'yearstart' and 'yearend' are the same
print(f"Are the columns datavalue and datavaluealt end the same?\n{filtered_asthma_df['datavalue'].equals(filtered_asthma_df['datavaluealt'])}")
print(f"Are the columns yearstart and year end the same?\n{filtered_asthma_df['yearstart'].equals(filtered_asthma_df['yearend'])}")
#if these columns are the same drop one of the duplicates and other not useful columns
if filtered_asthma_df['yearstart'].equals(filtered_asthma_df['yearend']):
    columns_to_drop = ['yearend',\
                        'topic',\
                        'datavaluealt',\
                        'topicid',\
                        'datavaluetypeid',\
                        'stratificationcategoryid1',\
                        'stratificationid1',\
                        'lowconfidencelimit',\
                       'highconfidencelimit',\
                       'datavaluefootnotesymbol',\
                      'datavaluefootnote']
    filtered_asthma_df=filtered_asthma_df.drop(columns=columns_to_drop)
    print(f"The following columns {columns_to_drop} were dropped")

Are the columns datavalue and datavaluealt end the same?
True
Are the columns yearstart and year end the same?
True
The following columns ['yearend', 'topic', 'datavaluealt', 'topicid', 'datavaluetypeid', 'stratificationcategoryid1', 'stratificationid1', 'lowconfidencelimit', 'highconfidencelimit', 'datavaluefootnotesymbol', 'datavaluefootnote'] were dropped


In [13]:
#renaming some columns
filtered_asthma_df=filtered_asthma_df.rename(columns={"yearstart": "year","locationabbr":"state_id","locationdesc":"state"})

In [14]:
#checking for missing data
filtered_asthma_df.isnull().sum()


year                          0
state_id                      0
state                         0
datasource                    0
question                      0
datavaluetype                 0
datavalue                  1602
stratificationcategory1       0
stratification1               0
locationid                    0
questionid                    0
datavalueunit                 0
dtype: int64

In [15]:
# removing missing data values
filtered_asthma_df=filtered_asthma_df[filtered_asthma_df['datavalue'].isna()==False]
filtered_asthma_df

,year,state_id,state,datasource,question,datavaluetype,datavalue,stratificationcategory1,stratification1,locationid,questionid,datavalueunit
19,2017,IA,Iowa,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,12,Gender,Female,19,AST1_1,%
20,2016,MO,Missouri,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,7,Gender,Male,29,AST1_1,%
22648,2016,AZ,Arizona,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,6.7,Race/Ethnicity,Hispanic,4,AST1_1,%
22649,2019,AL,Alabama,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,9.5,Overall,Overall,1,AST1_1,%
22650,2012,CO,Colorado,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,7.1,Gender,Male,8,AST1_1,%
...,...,...,...,...,...,...,...,...,...,...,...,...
37541,2012,ME,Maine,BRFSS,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,17.1,Race/Ethnicity,"Multiracial, non-Hispanic",23,AST1_1,%
37543,2016,NM,New Mexico,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,11.4,Race/Ethnicity,"White, non-Hispanic",35,AST1_1,%
37547,2019,ID,Idaho,BRFSS,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,7.9,Gender,Male,16,AST1_1,%
37550,2014,MO,Missouri,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,9.7,Overall,Overall,29,AST1_1,%


In [16]:
# checking that all the missing data value were correctly removed
filtered_asthma_df.isnull().sum()

year                       0
state_id                   0
state                      0
datasource                 0
question                   0
datavaluetype              0
datavalue                  0
stratificationcategory1    0
stratification1            0
locationid                 0
questionid                 0
datavalueunit              0
dtype: int64

In [17]:
filtered_asthma_df['question'].unique()

array(['Current asthma prevalence among adults aged >= 18 years'],
      dtype=object)

In [18]:
# drop territories (PR, GU, VI) and nation-wide data (US)
state_to_drop = ['PR','GU','US','VI']
filtered_asthma_df = filtered_asthma_df[~filtered_asthma_df['state_id'].isin(state_to_drop)]

filtered_asthma_df

,year,state_id,state,datasource,question,datavaluetype,datavalue,stratificationcategory1,stratification1,locationid,questionid,datavalueunit
19,2017,IA,Iowa,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,12,Gender,Female,19,AST1_1,%
20,2016,MO,Missouri,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,7,Gender,Male,29,AST1_1,%
22648,2016,AZ,Arizona,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,6.7,Race/Ethnicity,Hispanic,4,AST1_1,%
22649,2019,AL,Alabama,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,9.5,Overall,Overall,1,AST1_1,%
22650,2012,CO,Colorado,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,7.1,Gender,Male,8,AST1_1,%
...,...,...,...,...,...,...,...,...,...,...,...,...
37541,2012,ME,Maine,BRFSS,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,17.1,Race/Ethnicity,"Multiracial, non-Hispanic",23,AST1_1,%
37543,2016,NM,New Mexico,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,11.4,Race/Ethnicity,"White, non-Hispanic",35,AST1_1,%
37547,2019,ID,Idaho,BRFSS,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,7.9,Gender,Male,16,AST1_1,%
37550,2014,MO,Missouri,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,9.7,Overall,Overall,29,AST1_1,%


In [19]:
#checking for duplicates
filtered_asthma_df=filtered_asthma_df.drop_duplicates()
filtered_asthma_df

,year,state_id,state,datasource,question,datavaluetype,datavalue,stratificationcategory1,stratification1,locationid,questionid,datavalueunit
19,2017,IA,Iowa,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,12,Gender,Female,19,AST1_1,%
20,2016,MO,Missouri,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,7,Gender,Male,29,AST1_1,%
22648,2016,AZ,Arizona,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,6.7,Race/Ethnicity,Hispanic,4,AST1_1,%
22649,2019,AL,Alabama,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,9.5,Overall,Overall,1,AST1_1,%
22650,2012,CO,Colorado,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,7.1,Gender,Male,8,AST1_1,%
...,...,...,...,...,...,...,...,...,...,...,...,...
37541,2012,ME,Maine,BRFSS,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,17.1,Race/Ethnicity,"Multiracial, non-Hispanic",23,AST1_1,%
37543,2016,NM,New Mexico,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,11.4,Race/Ethnicity,"White, non-Hispanic",35,AST1_1,%
37547,2019,ID,Idaho,BRFSS,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,7.9,Gender,Male,16,AST1_1,%
37550,2014,MO,Missouri,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,9.7,Overall,Overall,29,AST1_1,%


In [20]:
#checking that there isn't more than one data value entry for the same year and state.
duplicateRowsDF = filtered_asthma_df[filtered_asthma_df.duplicated(subset=['year','state_id','state','datasource','question','datavaluetype','datavaluetype','stratificationcategory1','stratification1','locationid','questionid','datavalueunit'], keep=False)]
duplicateRowsDF

,year,state_id,state,datasource,question,datavaluetype,datavalue,stratificationcategory1,stratification1,locationid,questionid,datavalueunit


In [21]:
#keeping only the Overall values
filtered_asthma_df = filtered_asthma_df[filtered_asthma_df['stratificationcategory1'] == 'Overall']
filtered_asthma_df

,year,state_id,state,datasource,question,datavaluetype,datavalue,stratificationcategory1,stratification1,locationid,questionid,datavalueunit
22649,2019,AL,Alabama,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,9.5,Overall,Overall,1,AST1_1,%
22673,2012,FL,Florida,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,8.2,Overall,Overall,12,AST1_1,%
22676,2019,OH,Ohio,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,11.1,Overall,Overall,39,AST1_1,%
22681,2011,VT,Vermont,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,11.1,Overall,Overall,50,AST1_1,%
22697,2014,OK,Oklahoma,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,9.7,Overall,Overall,40,AST1_1,%
...,...,...,...,...,...,...,...,...,...,...,...,...
37440,2011,WV,West Virginia,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,9.2,Overall,Overall,54,AST1_1,%
37524,2013,ID,Idaho,BRFSS,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,8.7,Overall,Overall,16,AST1_1,%
37536,2019,NC,North Carolina,BRFSS,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,8.2,Overall,Overall,37,AST1_1,%
37550,2014,MO,Missouri,BRFSS,Current asthma prevalence among adults aged >=...,Crude Prevalence,9.7,Overall,Overall,29,AST1_1,%


In [22]:
# final dropping of unnecessary columns
filtered_asthma_df=filtered_asthma_df.drop(columns=['state',\
                                                    'datasource',\
                                                    'stratificationcategory1',\
                                                    'stratification1',\
                                                    'locationid',\
                                                    'questionid'])
filtered_asthma_df

,year,state_id,question,datavaluetype,datavalue,datavalueunit
22649,2019,AL,Current asthma prevalence among adults aged >=...,Crude Prevalence,9.5,%
22673,2012,FL,Current asthma prevalence among adults aged >=...,Crude Prevalence,8.2,%
22676,2019,OH,Current asthma prevalence among adults aged >=...,Crude Prevalence,11.1,%
22681,2011,VT,Current asthma prevalence among adults aged >=...,Crude Prevalence,11.1,%
22697,2014,OK,Current asthma prevalence among adults aged >=...,Crude Prevalence,9.7,%
...,...,...,...,...,...,...
37440,2011,WV,Current asthma prevalence among adults aged >=...,Crude Prevalence,9.2,%
37524,2013,ID,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,8.7,%
37536,2019,NC,Current asthma prevalence among adults aged >=...,Age-adjusted Prevalence,8.2,%
37550,2014,MO,Current asthma prevalence among adults aged >=...,Crude Prevalence,9.7,%


In [23]:
# adding collection about asthma to MongoDB
add_collection_to_mongo('asthma',filtered_asthma_df)

asthma stored in the MongoDB database named Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'health_air_mongo')


In [24]:
# adding the asthma table to the database and controlling that there are data recorded.
add_table_to_db('asthma',filtered_asthma_df, conn)

[(22649,
  '2019',
  'AL',
  'Current asthma prevalence among adults aged >= 18 years',
  'Crude Prevalence',
  '9.5',
  '%'),
 (22673,
  '2012',
  'FL',
  'Current asthma prevalence among adults aged >= 18 years',
  'Crude Prevalence',
  '8.2',
  '%'),
 (22676,
  '2019',
  'OH',
  'Current asthma prevalence among adults aged >= 18 years',
  'Crude Prevalence',
  '11.1',
  '%'),
 (22681,
  '2011',
  'VT',
  'Current asthma prevalence among adults aged >= 18 years',
  'Crude Prevalence',
  '11.1',
  '%'),
 (22697,
  '2014',
  'OK',
  'Current asthma prevalence among adults aged >= 18 years',
  'Crude Prevalence',
  '9.7',
  '%'),
 (22701,
  '2017',
  'ME',
  'Current asthma prevalence among adults aged >= 18 years',
  'Crude Prevalence',
  '11.2',
  '%'),
 (22703,
  '2011',
  'TX',
  'Current asthma prevalence among adults aged >= 18 years',
  'Crude Prevalence',
  '7.4',
  '%'),
 (22727,
  '2016',
  'AK',
  'Current asthma prevalence among adults aged >= 18 years',
  'Crude Prevalence'

# ETL of [Air Data](https://www.epa.gov/outdoor-air-quality-data)

In [ ]:
#finding the name of all the files in the air quality folder
all_files = glob.glob('./Air_quality_csv/*.csv')

air_list = []
#reading all the CSV files
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    air_list.append(df)
#creating the concatenated dataframe
air_df = pd.concat(air_list, axis=0, ignore_index=True)


In [ ]:
air_df

In [ ]:
# there is the need to aggregate the data by state and year
# The output is the average number of days that were over the law limits per parameter of interest (i.e. PMx)
# The average median and maximum API per state in a year is evaluated as well.
aggregate_air_df = air_df.groupby(by=['State','Year']).mean()
air_df = aggregate_air_df.reset_index()

In [ ]:
#printing the names of the dataframe columns
air_df.columns

In [ ]:
#dropping not useful column 
columns_to_drop = ['Days with AQI',\
                   'Good Days',\
                   'Moderate Days',\
                   'Unhealthy for Sensitive Groups Days',\
                   'Unhealthy Days',\
                   'Very Unhealthy Days',\
                   'Hazardous Days',\
                   '90th Percentile AQI',
                   'Days CO',\
                   'Days NO2',\
                   'Days SO2',]
air_df=air_df.drop(columns=columns_to_drop)

In [ ]:
air_df

In [ ]:
#converting the number of days in integers because as float they do not make too much sense
for column in air_df.columns:
    if air_df[column].dtype == float:
        air_df[column]= air_df[column].astype('int64')

In [ ]:
# adding collection about asthma to sqlite
add_table_to_db('air', air_df, conn)

In [ ]:
#removing the point from the key for handling the dataset in MongoDb without issues
air_df=air_df.rename(columns={"Days PM2.5": "Days PM2_5"})
air_df

In [ ]:
# adding collection about asthma to MongoDB
add_collection_to_mongo('air',air_df)

In [ ]:
#closing the connections to sqlite and MongoDB
conn.close()
mongo_client.close()